 # Best Practices for Preprocessing Natural Language Data
In this notebook, we improve the quality of our Project Gutenberg word vectors by adopting best-practices for preprocessing natural language data.

# Load dependencies

In [ ]:
import nltk
from nltk import word_tokenize, sent_tokenize
import gensim
from gensim.models.word2vec import Word2Vec
from sklearn.manifold import TSNE
import pandas as pd
from bokeh.io import output_notebook, output_file
from bokeh.plotting import show, figure
%matplotlib inline

In [ ]:
nltk.download('punkt')

In [ ]:
import string
from nltk.corpus import stopwords
from nltk.stem.porter import *
from gensim.models.phrases import Phraser, Phrases
from keras.preprocessing.text import one_hot

In [ ]:
nltk.download('stopwords')

# Load data

In [ ]:
nltk.download('gutenberg')

In [ ]:
from nltk.corpus import gutenberg
gberg_sents = gutenberg.sents()

# Iteratively preprocess a sentence:

In [ ]:
gberg_sents[4]

# to lowercase:

In [ ]:
[w.lower() for w in gberg_sents[4]]

# remove stopwords and punctuation:

In [ ]:
stpwrds = stopwords.words('english') + list(string.punctuation)

In [ ]:
stpwrds

In [ ]:
[w.lower() for w in gberg_sents[4] if w not in stpwrds]

# stem words:

In [ ]:
stemmer = PorterStemmer()

In [ ]:
[stemmer.stem(w.lower()) for w in gberg_sents[4] if w not in stpwrds]

# handle bigram collocations:

In [ ]:
phrases = Phrases(gberg_sents)

In [ ]:
bigram = Phraser(phrases)

In [ ]:
bigram.phrasegrams 

In [ ]:
"Jon lives in New York City".split()

In [ ]:
bigram["Jon lives in New York City".split()]

# Preprocess the corpus

In [ ]:
lower_sents = []
for s in gberg_sents:
    lower_sents.append([w.lower() for w in s if w not in list(string.punctuation)])

In [ ]:
lower_sents[0:5]

In [ ]:
lower_bigram = Phraser(Phrases(lower_sents))

In [ ]:
lower_bigram.phrasegrams

In [ ]:
lower_bigram["jon lives in new york city".split()]

In [ ]:
lower_bigram = Phraser(Phrases(lower_sents, min_count=32, threshold=64))
lower_bigram.phrasegrams

In [ ]:
clean_sents = []
for s in lower_sents:
    clean_sents.append(lower_bigram[s])

In [ ]:
clean_sents[0:9]

In [ ]:
clean_sents[6]

# Run word2vec

In [ ]:
model = Word2Vec(sentences=clean_sents, size=64, sg=1, window=10, min_count=10, seed=42, workers=8)
model.save('clean_gutenberg_model.w2v')

# Explore model

In [ ]:
model = gensim.models.Word2Vec.load('../input/dataset/clean_gutenberg_model.w2v')

In [ ]:
len(model.wv.vocab)

In [ ]:
model['ma_am']

In [ ]:
model.most_similar('ma_am') 

In [ ]:
model.most_similar(positive=['ma_am', 'man'], negative=['woman'])

In [ ]:
model.most_similar(positive=['father', 'woman'], negative=['man']) 

# Reduce word vector dimensionality with t-SNE

In [ ]:
tsne = TSNE(n_components=2, n_iter=1000)

In [ ]:
X_2d = tsne.fit_transform(model[model.wv.vocab])

In [ ]:
coords_df = pd.DataFrame(X_2d, columns=['x','y'])
coords_df['token'] = model.wv.vocab.keys()

In [ ]:
coords_df.head()

In [ ]:
coords_df.to_csv('clean_gutenberg_tsne.csv', index=False)

# Visualise

In [ ]:
coords_df = pd.read_csv('../input/dataset/clean_gutenberg_tsne.csv')

In [ ]:
_ = coords_df.plot.scatter('x', 'y', figsize=(12,12), marker='.', s=10, alpha=0.2)